# Assignment #2: Language models
Author: Pierre Nugues

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment

## Submission

Once you have written all the missing code and run all the cells, you will show it to an instructor that will pass you.

## Organization

* Each group will have to write Python programs to count unigrams, bigrams, and trigrams in a corpus of approximately one million words and to determine the probability of a sentence.
* You can test you regular expression using the regex101.com site
* Each student will have to write a short report of one to two pages and comment briefly the results. In your report, you must produce the tabulated results of your analysis as described below.

## Programming

### Imports

In [549]:
import bz2
import math
import os
import regex as re
import requests
import sys
from zipfile import ZipFile
import numpy as np
from collections import defaultdict

Some imports you may need. Add others as needed.

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [586]:
# You may have to adjust the path
corpus = open('Selma.txt', encoding='utf8').read()

In [550]:
pattern = 'Nils Holgersson'
width = 25

In [587]:
# spaces match tabs and newlines
pattern = re.sub(' ', r'\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub(r'\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, clean_corpus):
    print(match.group(1))
# print the string with 0..width characters on either side

Selma Lagerlöf Nils Holgerssons underbara resa genom Sv
! Se på Tummetott! Se på Nils Holgersson Tummetott!» Genast vände
r,» sade han. »Jag heter Nils Holgersson och är son till en husma
lden. »Inte är det värt, Nils Holgersson, att du är ängslig eller
 i dem. På den tiden, då Nils Holgersson drog omkring med vildgäs
ulle allt visa honom vad Nils Holgersson från Västra Vemmenhög va
om ägde rum det året, då Nils Holgersson for omkring med vildgäss
m vad det kan kosta dem. Nils Holgersson hade inte haft förstånd 
de det inte mer sägas om Nils Holgersson, att han inte tyckte om 
 Rosenbom?» För där stod Nils Holgersson mitt uppe på Rosenboms n
 Med ens fingo de syn på Nils Holgersson, och då sköt den store v
vila. När vildgässen och Nils Holgersson äntligen hade letat sig 
 slags arbetare. Men vad Nils Holgersson inte såg, det var, att s
nde han fråga, och om då Nils Holgersson sade nej, började han ge
de lille Mats, och om nu Nils Holgersson också hade tegat, så had
åg så försmädlig ut,

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [551]:
def tokenize(text):
    words = re.findall(r'\p{L}+', text)
    return words

In [588]:
words = tokenize(corpus)
words[:10]

['Selma',
 'Lagerlöf',
 'Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Första',
 'bandet']

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [589]:
# Write your code here
len(set(words))

44256

Lowercased text

In [592]:
# Write your code here
lower = set(map(lambda x: x.lower(), words))
len(lower)

41032

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

In [553]:
# Write your code
nonletter = r"[^\p{L}.;:?!]+"


Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [593]:
# Write your code here
def clean(text):
  return re.sub(nonletter, " ", text)

In [594]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

In [595]:
test_para = clean(test_para)
test_para

'En gång hade de på Mårbacka en barnpiga som hette Back Kajsa. Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.'

In [597]:
print(corpus[:200])

Selma Lagerlöf


Nils Holgerssons underbara resa genom Sverige. Första bandet





Bokutgåva


Albert Bonniers förlag, Stockholm 1907.




DEN KRISTLIGA DAGVISAN.





Den signade dag, som vi nu här s


In [596]:
print(clean(corpus[:200]))

Selma Lagerlöf Nils Holgerssons underbara resa genom Sverige. Första bandet Bokutgåva Albert Bonniers förlag Stockholm . DEN KRISTLIGA DAGVISAN. Den signade dag som vi nu här s


#### Segmenter

##### Detecting sentence boundaries

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces. Do not use `\s`.

In [598]:
# Write your code here
sentence_boundaries = r"(\p{P})(\p{Z}*)(\p{Lu})"

##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [599]:
# Write your code here
sentence_markup = r" </s>\n<s> \3"

##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

Insert markup codes in the beginning and end of the text

In [600]:
# Write your code here
text = re.sub(sentence_boundaries, sentence_markup, test_para)

In [601]:
print(text)

En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.


In [581]:
# Write your code here
formated_text = f"<s> {text}</s>"
result = re.sub(sentence_boundaries, sentence_markup, formated_text)

In [602]:
print(result)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.</s>


Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex. Do not use `\s`.

In [582]:
# Write your code here
final_text = re.sub(r"(?![/])\p{P}", " ", result)

In [603]:
print(final_text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [604]:
# Write your code here
def segment_sentences(text):
  text = re.sub(sentence_boundaries, sentence_markup, text)
  text = f"<s> {text}</s>"
  text = re.sub(r"(?![/])\p{P}", " ", text)
  text = re.sub(r'\p{Z}+', " ", text)
  return text.lower()

In [605]:
print(segment_sentences(test_para))

<s> en gång hade de på mårbacka en barnpiga som hette back kajsa </s>
<s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Estimate qualitatively the accuracy of your program.

#### Tokenizing the corpus

In [584]:
# Write your code here
corpus_clean = clean(corpus)
corpus_segmented = segment_sentences(corpus_clean)

In [585]:
print(corpus_segmented[-557:])

<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s>
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s>


You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [639]:
# Write your code here
words = re.split(r"[ \n]+", corpus_segmented)

The five last lines of the corpus should like this:

In [640]:
print(words[-101:])

['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']


Read and try programs to compute the frequency of unigrams and bigrams of the training set: [<a
            href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>].

#### Unigrams

In [641]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency

We compute the frequencies.

In [654]:
frequency = unigrams(words)
list(frequency.items())[:20]

[('<s>', 59052),
 ('selma', 52),
 ('lagerlöf', 270),
 ('nils', 87),
 ('holgerssons', 6),
 ('underbara', 23),
 ('resa', 317),
 ('genom', 688),
 ('sverige', 56),
 ('</s>', 59052),
 ('första', 525),
 ('bandet', 6),
 ('bokutgåva', 11),
 ('albert', 15),
 ('bonniers', 11),
 ('förlag', 11),
 ('stockholm', 77),
 ('den', 11624),
 ('kristliga', 2),
 ('dagvisan', 2)]

In [643]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams

In [655]:
frequency_bigrams = bigrams(words)
list(frequency_bigrams.items())[:20]

[(('<s>', 'selma'), 8),
 (('selma', 'lagerlöf'), 11),
 (('lagerlöf', 'nils'), 1),
 (('nils', 'holgerssons'), 6),
 (('holgerssons', 'underbara'), 4),
 (('underbara', 'resa'), 4),
 (('resa', 'genom'), 6),
 (('genom', 'sverige'), 5),
 (('sverige', '</s>'), 17),
 (('</s>', '<s>'), 59051),
 (('<s>', 'första'), 11),
 (('första', 'bandet'), 1),
 (('bandet', 'bokutgåva'), 2),
 (('bokutgåva', 'albert'), 11),
 (('albert', 'bonniers'), 11),
 (('bonniers', 'förlag'), 11),
 (('förlag', 'stockholm'), 10),
 (('stockholm', '</s>'), 24),
 (('<s>', 'den'), 1375),
 (('den', 'kristliga'), 2)]

With a given number of words in a corpus, some expression could give the number of pairs that are able to be created. 

### Computing the likelihood of a sentence
You will now compute the likelihood of a sentence using a bigram and a trigram models. 

In both models, you will ignore the start of sentence symbol, `<s>`, as this factor is common to both models: $P(<s>)$. This will save you one multiplication.

Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [<a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>]. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_.

Your figures might be slightly different because of differences in the sentence segmentation.

In [630]:
# Write your code
def unigram_lm(frequency, sent_words):
    total_words = sum(frequency.values()) # - frequency['<s>']
    
    probs = [frequency[word] / total_words for word in sent_words]
    unigram_prob = np.prod(probs)   
    geo_mean = math.pow(unigram_prob, 1 / len(probs))
    entropy_rate = -1 * np.log2(unigram_prob) / len(sent_words)
    perplexity = math.pow(2, entropy_rate)
    
    print(f"Unigram probability: {unigram_prob}")
    print(f"Geometric mean probability: {geo_mean}")
    print(f"Entropy rate: {entropy_rate}")
    print(f"Perplexity: {perplexity}")
    return perplexity


In [648]:
sentence = 'det var en gång en katt som hette nils </s>'
sent_words = sentence.split()
sent_words

['det', 'var', 'en', 'gång', 'en', 'katt', 'som', 'hette', 'nils', '</s>']

In [649]:
perplexity_unigrams = unigram_lm(frequency, sent_words)

Unigram probability: 5.364076143769206e-27
Geometric mean probability: 0.002360203735618772
Entropy rate: 8.726872884532442
Perplexity: 423.6922367796484


In [650]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams

423

#### Bigrams

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [663]:
# Write your code
def bigram_lm(frequency, frequency_bigrams, sent_words):
    total_words = sum(frequency.values())
    probabilities = []
    for i, word in enumerate(sent_words):
        
        if not sent_words.index(word) + 1 >=  len(sent_words):
            if (word, sent_words[i + 1]) in frequency_bigrams:
                probabilities.append(frequency_bigrams[(word, sent_words[i + 1])] / frequency[word])
            else:
                probabilities.append(frequency[sent_words[i + 1]] / total_words)
        # print(word, sent_words[i+1], probabilities[i], frequency[word], frequency_bigrams[(word, sent_words[i + 1])])
    
    prob_bigram = np.prod(probabilities)  
    geo_mean = np.power(prob_bigram, 1 / len(probabilities))
    entropy_rate = -np.log2(prob_bigram) / len(probabilities)
    perplexity = math.pow(2, entropy_rate)
    
    print(f"Bigram probability: {prob_bigram}")
    print(f"Geometric mean probability: {geo_mean}")
    print(f"Entropy rate: {entropy_rate}")
    print(f"Perplexity: {perplexity}")
    return perplexity
    


In [664]:
perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, ["<s>"] + sent_words)

Bigram probability: 2.375902423886631e-19
Geometric mean probability: 0.013727228410269554
Entropy rate: 6.186815821565451
Perplexity: 72.84791730076296


In [665]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

72

In addition to this sentence, _Det var en gång en katt som hette Nils_, write three other sentences that will form your test set and run your programs on them. You will insert them in your report.

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

In [617]:
# Write your code
def trigrams(words):
  trigrams = [tuple(words[idx:idx + 3])
                for idx in range(len(words) - 2)]
  frequencies = {}
  for trigram in trigrams:
      if trigram in frequencies:
          frequencies[trigram] += 1
      else:
          frequencies[trigram] = 1
  return frequencies

In [618]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

330

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [619]:
starting_text = 'De'.lower()
starting_text

'de'

In [620]:
cand_nbr = 5

In [666]:
# Write your code here
current_word_predictions_1 = []
candidates = defaultdict(int)
for words, count in frequency_bigrams.items():
    if words[0] == "<s>" and words[1].startswith(starting_text):
        candidates[words] = count

candidates = sorted(candidates.items(), key=lambda x: (-x[1], x[0]))[:cand_nbr]
current_word_predictions_1 = [word[0][1] for word in candidates]

Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [667]:
current_text = "Det var en ".lower()
current_text

'det var en '

In [623]:
# Write your code here
tokens = tokenize(current_text)

In [668]:
tokens

['det', 'var', 'en']

In [625]:
# Write your code here
next_word_candidates = defaultdict(int)
# if len(tokens) > 2:
for (word1, word2, word3), count in frequency_trigrams.items():
    if word1 == tokens[-2] and word2 == tokens[-1]:
        next_word_candidates[word3] += count
  
next_word_candidates = sorted(next_word_candidates.items(), key=lambda x: (-x[1], x[0]))[:cand_nbr]
next_word_predictions = [word[0] for word in next_word_candidates]


In [626]:
next_word_predictions

['stor', 'liten', 'gammal', 'god', 'sådan']

In [627]:
current_text = "Det var en g".lower()

In [628]:
# Write your code here
current_word_candidates_2 = defaultdict(int)
*first_words, last_word = tokenize(current_text)
for (word1, word2, word3), count in frequency_trigrams.items():
    if word1 == first_words[-2] and word2 == first_words[-1] and word3.startswith(last_word):
        current_word_candidates_2[word3] += count

current_word_candidates_2 = sorted(current_word_candidates_2.items(), key=lambda x: (-x[1], x[0]))[:cand_nbr]
current_word_predictions_2 = [word[0] for word in next_word_candidates_2]

In [629]:
current_word_predictions_2

['gammal', 'god', 'gång', 'ganska', 'glädje']

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. You will include the __regular expression you used to segment the text__ and the __unigram and bigram tables__ for _Det var en gång en katt som hette Nils_ and __two other sentences__.
2. Execute the Jupyter notebook by Peter Norvig here: <a href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">https://nbviewer.jupyter.org/url/norvig.com/ipython/How to Do Things with Words.ipynb</a>. Just run all the cells and be sure that you understand the code. You will find the data here: <a href="http://norvig.com/ngrams/">http://norvig.com/ngrams/</a>.
3. In your report, after the description of your program, you will describe one experiment with Norvig's notebook and a __long string of words your will create yourself or copy from a text you like__. You will remove all the punctuation and white spaces from this string. You will set this string in lowercase letters. You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. __You will comment the segmentation results you obtained__ with the unigram and bigram models.

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is September 22, 2023. You will have only three submission attempts. The deadline for the second and third ones are one week after you are noticed of your result.